Análisis REM8
 ==========

Comienzo cargando todos los paquetes necesarios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime as dt
import datetime
from datetime import timedelta  
import timeboard as tb # turnos 

# para tratar de poner espanol
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time, format_timedelta, Locale
import locale                                    # para tratar de poner espanol
#locale = Locale('es', 'CL')
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')  

import os
import runpy
from pyexcel_ods import get_data
from matplotlib_venn import venn3
from matplotlib_venn import venn2, venn2_circles
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


#otro graficador
import seaborn as sns

#stats

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/home/egidio/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# cargo algunos scripts que me suelen servir para análisis

In [3]:
#listo los archivos
scripts = []
for dirname, dirnames, filenames in os.walk('/home/egidio/Dropbox/Compucosas/programas py'):
    for filename in filenames:
        scripts.append(os.path.join(dirname, filename))

scripts

# los cargo
for script in scripts:
    if script[-3:] == '.py':
        exec(open(str(script)).read())


['/home/egidio/Dropbox/Compucosas/programas py/lector_ODF.py',
 '/home/egidio/Dropbox/Compucosas/programas py/como importar',
 '/home/egidio/Dropbox/Compucosas/programas py/descripcion_univariada.py',
 '/home/egidio/Dropbox/Compucosas/programas py/características_operativas_de_test.py',
 '/home/egidio/Dropbox/Compucosas/programas py/percentiles.py',
 '/home/egidio/Dropbox/Compucosas/programas py/tabala_frecuencias.py',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/percentiles.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/lector_ODF.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/descripcion_univariada.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/características_operativas_de_test.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/tabala_frecuencias.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/funciones_locales.cpython-36.pyc']

# Cargo la BD

In [4]:
BD = pd.read_pickle('./BD/eSAMU.pkl')

In [5]:
BD.columns
BD.head(2)

Index(['Id', 'Fecha', 'Motivo del Llamado', 'Submotivo del Llamado',
       'Prioridad', 'Estado', 'Edad', 'Género', 'Dirección', 'Sector',
       'Comuna', 'Región', 'Intervención', 'Tipo Traslado',
       'Categoría Vehículo', 'Condición Traslado', 'En Regulación',
       'En Despacho', 'En Curso', 'Atendida', 'Completado', 'Regulado',
       'Usuario que Cierra', 'Móvil Agregado', 'Móvil Despachado',
       'Móvil Salida', 'Móvil Llegada Lugar', 'Móvil en Traslado',
       'Móvil Llegada a CA', 'Móvil Recepción Paciente',
       'Móvil Camilla Retenida', 'Móvil Vuelta a Base', 'Móvil LLegada a Base',
       'Móvil Finalizado', 'Nombre Vehículo', 'Tipo Requerido',
       'Tipo Despachado', 'Km Recorrido', 'Chofer', 'Paramédico', 'Enfermero',
       'Doctor', 'Longitud', 'Latitud', 'Diff_recep_Despachado',
       'Diff_Despacho_Salida', 'Diff_salida_enellugar', 'Diff_z6_1',
       'Diff_z6_2', 'Diff_z6_3'],
      dtype='object')

,Id,Fecha,Motivo del Llamado,Submotivo del Llamado,Prioridad,Estado,Edad,Género,Dirección,Sector,...,Enfermero,Doctor,Longitud,Latitud,Diff_recep_Despachado,Diff_Despacho_Salida,Diff_salida_enellugar,Diff_z6_1,Diff_z6_2,Diff_z6_3
0,131301,2020-01-31 23:58:13,Accidentes y Traumas,Caídas leves,S2,Completado,40.0,Femenino,arlegui con quillota,centro,...,Karen Urra coloma,NaN,NaN,NaN,03:03:22,00:01:11,00:03:37,NaT,NaT,NaT
1,131300,2020-01-31 23:29:08,Consciencia,Comportamientos extraños,NaN,Completado,63.0,Femenino,estacion metro,c9,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT


para efectos de este análisis es importante dividir ambulancias según su complejidad

In [6]:
BD['Bas_avan'] = BD['Tipo Despachado'] 
BD.loc[BD['Tipo Despachado'] == 'm1' , 'Bas_avan' ] = 'Básico'
BD.loc[BD['Tipo Despachado'] == 'm2' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'm3' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'x5' , 'Bas_avan' ] = 'Avanzado'

defino Bas_avan como categórico

# defino la BD con la que trabajaré

El informe a continuación es un análisis del:
* fecha
* sector

## según fecha

In [7]:
rango = (                                            #mes Enero
    (BD.Fecha > pd.to_datetime('2020-01-01')) &
    (BD.Fecha < pd.to_datetime('2020-01-31'))
)


db1 = BD[rango].copy()
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2020-01', 'M')], dtype=object)

## segun ambulancias

In [8]:
db1['Nombre Vehículo'].unique()

array([nan, 'R23', 'R31', 'R24', 'R21', 'AP16', 'AP1', 'R20', 'R32',
       'R83', 'R84', 'R42', 'R11', 'R62', 'R10', 'R80', 'AT90', 'AP28',
       'R25', 'R68', 'Móvil Loncura', 'Móvil Hijuelas',
       'Móvil Las Parcelas', 'R14', 'R41', 'AP26', 'AP15',
       'Móvil Artificio', 'Móvil Puchuncaví', 'AP 14', 'AP27', 'R71',
       'AP31', 'MOVIL PAPUDO', 'R73', 'nogales movil 8', 'AP51', 'R57',
       'MOVIL LONCURA', 'R72', 'W133 Carabineros', 'Móvil Los Molles',
       'AP41', 'R52', 'R13', 'R53', 'Móvil Ventana', 'W56', 'R91', 'R35',
       'AP2', 'R92', 'sapu zapallar', 'R102', 'R44', 'Emecar 1',
       'MOVIL SAPU VENTANA', 'MOVIL 8 SAMU LITORAL', 'Móvil Nogales',
       'Móvil Papudo', 'movil rema', 'Movil Posta Catapilco',
       'MOVIL RUTA', 'Movil SAPU La Calera El trigal', 'R33', 'R22',
       'R61', 'R51', 'MOVIL SAMU ACONCAGUA', 'MOVIL CATAPILCO', 'R100'],
      dtype=object)

elegir uno

In [9]:
#elegir uno

sector = db1['Nombre Vehículo'].str.contains('R3') == True
db1 = db1[sector]

In [10]:
texto1 = 'Este es un inforque que considera el mes de '
texto2 = 'En el territorio de '

# sector = 'SSVQ'
sector = db1.Comuna.value_counts().index[0]

mes= db1.Fecha.describe()[4].strftime('%B')

result = texto1 + mes + '. ' + texto2 + sector + '.' + '\n \n'
result
print(result, file=open('Informe.txt','a'))

'Este es un inforque que considera el mes de enero. En el territorio de Quilpué.\n \n'

In [11]:
db1.Comuna.value_counts()

Quilpué          225
Villa Alemana     12
Viña del Mar       1
Name: Comuna, dtype: int64

# defino el archivo donde guardaré la info

Será un archivo

como dividir por fecha

In [12]:
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2020-01', 'M')], dtype=object)

## Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)

prestaciones según recurso

In [13]:
# los tiempos de z17 al z8 los divido según requisito planilla
cortado = pd.cut(
    db1.Diff_salida_enellugar.dt.seconds,
    [0, 20, 40, 1000000],
    labels = ['Menos 20 min',  '20 a 40 min',  'Más 40 min'])

# tabulo
#pd.crosstab(db1.Bas_avan, cortado, margins=True, margins_name= 'Total')

# tabulo por mes
print('Por Mes')
dum1 = cortado
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    result = pd.crosstab(
        test['Bas_avan'],
        dum1,
        margins=True, margins_name= 'Total', dropna=False)
    result.columns.name = 'Tiempo llegada al lugar'
    result.index.name = 'Tipo Móvil'
    result

Por Mes


Period('2020-01', 'M')

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,1,1,92,146
Básico,1,0,178,370
Total,3,1,328,615


escribo en el archivo

In [14]:
titulo = 'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'
texto = 'Se asume que todas las intervenciones críticas son realizadas \
por móviles avanzados y que las no-críticas por móviles básicos. Esto \
determina que las casillas de las prestaciones críticas tengan valores \
de cero para móviles básicos y las prestaciones no-críticas tengan \
valores cero para móviles avanzados. \n \n'
espacios = '\n \n'

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'

'Se asume que todas las intervenciones críticas son realizadas por móviles avanzados y que las no-críticas por móviles básicos. Esto determina que las casillas de las prestaciones críticas tengan valores de cero para móviles básicos y las prestaciones no-críticas tengan valores cero para móviles avanzados. \n \n'

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,1,1,92,146
Básico,1,0,178,370
Total,3,1,328,615


## Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA (Desde el lugar del evento a unidad de Emergencia)

In [15]:
db1['Categoría Vehículo'].value_counts()
#pd.crosstab(db1['Categoría Vehículo'], test['Bas_avan'])

db1['Categoría Vehículo'] = pd.Categorical(db1['Categoría Vehículo'],
                                           categories=['samu', 'enrutado', 'nosamu'])

enrutado    615
Name: Categoría Vehículo, dtype: int64

In [16]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] != 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2020-01', 'M')

Categoría Vehículo  Tipo móvil
enrutado            Avanzado      84
                    Básico        90
Name: Id, dtype: int64

escribo en el archivo

In [17]:
titulo = ' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'

' '

Categoría Vehículo  Tipo móvil
enrutado            Avanzado      84
                    Básico        90
Name: Id, dtype: int64

## Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)

In [18]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] == 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2020-01', 'M')

Categoría Vehículo  Tipo móvil
enrutado            Avanzado       62
                    Básico        280
Name: Id, dtype: int64

In [19]:
titulo = ' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

' '

Categoría Vehículo  Tipo móvil
enrutado            Avanzado       62
                    Básico        280
Name: Id, dtype: int64

# Sección N: CLASIFICACION DE LAS INTERVENCIONES POR GRANDES GRUPOS DE DIAGNOSTICOS (SAMU)

In [20]:
db1['Patol'] = np.nan
db1.loc[db1['Submotivo del Llamado'] == 'Accidente vehicular o transporte. Atropellos',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Accidente múltiples víctimas (cualquier razón)',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Caídas graves',
        'Patol'] = 'PoliTMT'

db1.loc[db1['Submotivo del Llamado'] == 'Colapso respiratorio o circulatorio. PCR. Asfixia',
        'Patol'] = 'PCR'

db1.loc[db1['Submotivo del Llamado'] == 'Dolor de pecho',
        'Patol'] = 'SCA'

db1.loc[db1['Patol'].isnull(),
        'Patol'] = 'Otros'

db1['Patol'] = pd.Categorical(db1['Patol'], categories=['SCA', 'PCR', 'PoliTMT', 'Otros'])

In [21]:
db1.loc[db1['Género'] == 'Femenino', 'Género'] = 'Feme'
db1.loc[db1['Género'] == 'Masculino', 'Género'] = 'Masc'

db1['Género'] = pd.Categorical(db1['Género'], categories=['Masc', 'Feme'])

In [22]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    dum1 = pd.cut(test.Edad, range(0, 90, 5), right=False)  
    test['Patol'].value_counts(sort= False)
    result0 = pd.crosstab(test['Patol'], test['Género'])
    separador = ' -----------------------------------------------------------------'
    result1 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,:8]
    result2 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,8:16]
    result3 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,16:24]
    result4 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,24:]
    result0
    result1
    result2
    result3
    result4

Period('2020-01', 'M')

SCA          8
PCR          6
PoliTMT     42
Otros      559
Name: Patol, dtype: int64

Género,Masc,Feme
Patol,,
SCA,4,4
PCR,2,3
PoliTMT,25,11
Otros,247,294


Edad    [0, 5)      [5, 10)      [10, 15)      [15, 20)     
Género    Masc Feme    Masc Feme     Masc Feme     Masc Feme
Patol                                                       
SCA          0    0       0    0        0    0        0    0
PCR          0    0       0    0        0    0        0    0
PoliTMT      0    0       3    0        2    0        1    0
Otros        6    7       8    5       10    6        8    8

Edad    [20, 25)      [25, 30)      [30, 35)      [35, 40)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            0    0        0    1        0    0        0    0
PCR            0    0        0    0        0    0        0    1
PoliTMT        1    1        3    2        3    3        1    3
Otros          7    6       13   13        8    6       11   21

Edad    [40, 45)      [45, 50)      [50, 55)      [55, 60)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            0    0        1    0        0    1        0    0
PCR            0    0        0    0        0    0        2    0
PoliTMT        2    0        2    0        1    1        2    0
Otros         17    8        9   11       11    9       28   26

Edad    [60, 65)      [65, 70)      [70, 75)      [75, 80)      [80, 85)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                                        
SCA            0    0        0    1        0    1        1    0        0    0
PCR            0    0        0    0        0    0        0    1        0    1
PoliTMT        1    1        1    0        0    0        1    0        0    0
Otros         14   12        9   14       28   40       18   23        9   21

In [23]:
titulo = ' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = 'Es una tabla larga, por lo que se divide en varios espacios '

titulo
texto
#result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result0, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result1, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result2, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result3, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result4, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(espacios, file=open('Informe.txt','a'))

' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

'Es una tabla larga, por lo que se divide en varios espacios '

# Finalización
Fin informe REM8

In [24]:
print('Fin informe REM 8', file=open('Informe.txt','a'))


# Análisis posteriores extra

In [25]:
db1[db1['Motivo del Llamado']=='Otro'][['Submotivo del Llamado', 'Id']]

,Submotivo del Llamado,Id
172,Fallecido,131061
478,Fallecido,130618
580,Fallecido,130488
1008,Fallecido,129954
1253,Fallecido,129619
1264,Fallecido,129558
1285,Fallecido,129519
1405,Fallecido,129376
1500,Fallecido,129230
1961,Fallecido,128512


In [26]:
BD.Fecha.describe()

count                  105517
unique                  91842
top       2018-11-23 14:30:10
freq                      442
first     2018-01-15 08:45:15
last      2020-01-31 23:58:13
Name: Fecha, dtype: object